<a href="https://colab.research.google.com/github/julioclerio/sistema_bancario_eng_dados_ada/blob/main/jupyter_notebook%20/sistema_bancario_eng_dados_ada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
from datetime import datetime, timedelta
from pytz import timezone
from typing import List, Dict, Optional
import random
import json
import random

In [ ]:
registros = []
id_counter = 1

In [ ]:
def gerar_registro_ficticio():

  """
    Generates a fictitious financial record.

    Returns:
    dict: A dictionary representing a financial record with the following keys:
        - 'id': Unique identifier for the record.
        - 'data': Date and time of the record in the format "%d/%m/%Y %H:%M".
        - 'dia': Day of the month.
        - 'mes': Month.
        - 'ano': Year.
        - 'tipo': Type of transaction ('receita', 'despesa', 'investimento').
        - 'valor': Transaction amount (positive for 'receita', negative for 'despesa').
        - 'montante': Investment amount (if the type is 'investimento', otherwise None).
        - 'taxa_juros': Interest rate (if the type is 'investimento', otherwise None).
        - 'rendimento': Investment return (initialized to 0).
    """

    tipos = ['receita', 'despesa', 'investimento']
    tipo = random.choice(tipos)

    if tipo == 'investimento':
        montante = round(random.uniform(1000, 10000), 2)
        taxa_juros = round(random.uniform(0.01, 0.1), 4)
    else:
        montante = None
        taxa_juros = None

    data_hora = datetime.now() - timedelta(days=random.randint(1, 365))

    registro = {
        'id': id_counter,
        'data': data_hora.strftime("%d/%m/%Y %H:%M"),
        'dia': data_hora.day,
        'mes': data_hora.month,
        'ano': data_hora.year,
        'tipo': tipo,
        'valor': round(random.uniform(100, 1000), 2) * (-1 if tipo == 'despesa' else 1),
        'montante': montante,
        'taxa_juros': taxa_juros,
        'rendimento': 0
    }

    return registro


In [ ]:
def criar_registro(tipo, valor, taxa_juros=None, montante = None):

   """
    Creates a financial record and adds it to the list of records.

    Args:
    - tipo (str): Type of transaction ('receita', 'despesa', 'investimento').
    - valor (float): Transaction amount.
    - taxa_juros (float, optional): Interest rate for 'investimento' type (default is None).
    - montante (float, optional): Investment amount for 'investimento' type (default is None).

    Returns:
    None
    """


    global id_counter
    data_hora = datetime.now()

    if tipo == 'investimento':
        montante = valor
        taxa_juros = float(input("Taxa de juros (% ao ano): "))

    registro = {
        'id': id_counter,
        'data': data_hora.strftime("%d/%m/%Y %H:%M"),
        'dia': data_hora.day,
        'mes': data_hora.month,
        'ano': data_hora.year,
        'tipo': tipo,
        'valor': valor if tipo != 'despesa' else -valor,
        'montante': montante,
        'taxa_juros': taxa_juros,
        'rendimento': 0
    }

    registros.append(registro)
    id_counter += 1
    atualiza_rendimento()
    salvar_no_csv(registro)

In [ ]:
def salvar_no_csv(registro):
    registro_bancario = 'registros.csv'
    try:
        with open(registro_bancario, mode='a', newline='') as arquivo_csv:
            aux = csv.writer(arquivo_csv, delimiter=',', quotechar='"')
            aux.writerow([registro['dia'], registro['mes'], registro['ano'],
                          registro['tipo'], registro['valor'], registro['montante'], registro['taxa_juros'], registro['rendimento'], registro['id']])
        print("Dados salvos com sucesso.")
    except FileNotFoundError:
        print(f"Erro: O arquivo {registro_bancario} não existe.")

In [ ]:
def ler_registros():
    print("Escolha o tipo de filtro:")
    print("1. Data")
    print("2. Tipo")
    print("3. Valor")
    print("4. Visualizar todos os registros")
    opcao_filtro = input("Escolha uma opção (1-3): ")

    if opcao_filtro == '1':
        print("Escolha a opção de visualização de data:")
        print("1. Mês")
        print("2. Dia")
        print("3. Ano")
        opcao_data = input("Escolha uma opção (1-3): ")

        if opcao_data == '1':
            filtro_data = input("Filtrar por mês (MM/AAAA): ")
        elif opcao_data == '2':
            filtro_data = input("Filtrar por dia (DD/MM/AAAA): ")
        elif opcao_data == '3':
            filtro_data = input("Filtrar por ano (AAAA): ")
        else:
            print("Opção inválida.")
            return

        encontrou_registros = False
        for registro in registros:
            if (opcao_data == '1' and registro['mes'] == int(filtro_data[:2]) and registro['ano'] == int(filtro_data[3:])) or \
               (opcao_data == '2' and registro['dia'] == int(filtro_data[:2]) and registro['mes'] == int(filtro_data[3:5]) and registro['ano'] == int(filtro_data[6:])) or \
               (opcao_data == '3' and registro['ano'] == int(filtro_data)):
                print(registro)
                encontrou_registros = True

        if not encontrou_registros:
            print("Nenhum registro encontrado.")

    elif opcao_filtro == '2':
        filtro_tipo = input("Filtrar por tipo (Receita/Despesa/Investimento): ").lower()
        encontrou_registros = False
        for registro in registros:
            if registro['tipo'] == filtro_tipo:
                print(registro)
                encontrou_registros = True

        if not encontrou_registros:
            print("Nenhum registro encontrado.")

    elif opcao_filtro == '3':
        filtro_valor = float(input("Filtrar por valor: "))
        encontrou_registros = False
        for registro in registros:
            if registro['valor'] == filtro_valor:
                print(registro)
                encontrou_registros = True

        if not encontrou_registros:
            print("Nenhum registro encontrado.")

    elif opcao_filtro == '4':

        for registro in registros:
            print(registro)

    else:
        print("Opção inválida.")


In [ ]:
def atualizar_registro(registro_id, novo_valor, novo_tipo, nova_data):
    for registro in registros:
        if registro['id'] == registro_id:
            registro['valor'] = novo_valor
            registro['tipo'] = novo_tipo
            registro['data'] = nova_data
            atualiza_rendimento()
            salvar_no_csv(registro)
            print(f"Registro {registro_id} atualizado com sucesso.")
            return
    print(f"Registro com ID {registro_id} não encontrado.")

In [ ]:
def deletar_registro(registro_id):
    global registros
    registros = [registro for registro in registros if registro['id'] != registro_id]
    atualiza_rendimento()
    salvar_no_csv({})
    print(f"Registro {registro_id} deletado com sucesso.")

In [ ]:
def atualiza_rendimento():
    for registro in registros:
        if registro['tipo'] == 'investimento':
            tempo = (datetime.now() - datetime.strptime(registro['data'], "%d/%m/%Y %H:%M")).days
            montante_atualizado = float(registro['montante']) * (1 + float(registro['taxa_juros']) / 100) ** (tempo / 365)
            registro['montante'] = montante_atualizado

In [ ]:
def exportar_relatorio(formato='csv'):
    if formato == 'csv':
        caminho_arquivo = 'relatorio.csv'
        with open(caminho_arquivo, mode='w', newline='') as arquivo_csv:
            escritor = csv.writer(arquivo_csv)
            escritor.writerow(['dia', 'mes', 'ano', 'tipo', 'valor', 'montante', 'taxa_juros', 'rendimento', 'id'])
            for registro in registros:
                escritor.writerow([registro['dia'], registro['mes'], registro['ano'],
                                   registro['tipo'], registro['valor'], registro['montante'], registro['taxa_juros'], registro['rendimento'], registro['id']])
        print(f'Relatório exportado para {caminho_arquivo}.')
    elif formato == 'json':
        caminho_arquivo = 'relatorio.json'
        with open(caminho_arquivo, mode='w') as arquivo_json:
            json.dump(registros, arquivo_json, indent=2)
        print(f'Relatório exportado para {caminho_arquivo}.')
    else:
        print("Formato não suportado.")


In [ ]:
def apresentacao (hora):
  if hora > 12 and hora < 18:
    return 'Boa Tarde!'
  elif hora > 18 and hora < 24:
    return 'Boa Noite!'
  elif hora > 24 and hora < 6:
    return 'Fora do horário de funcionamento!'
  elif hora > 6 and hora < 12:
    return 'Bom dia!'

In [ ]:
def data_hora():
  fuso_horario_sp = timezone("Brazil/East")
  hoje = datetime.now(fuso_horario_sp)
  hoje_dict = {'dia':hoje.day, 'mes':hoje.month, 'ano':hoje.year, 'hora':hoje.hour,'minuto':hoje.minute}
  return hoje_dict

In [ ]:
def agrupar_por(informacao):
    if informacao == 'mes':
        agrupar_por_mes()
    elif informacao == 'tipo':
        agrupar_por_tipo()
    elif informacao == 'valor':
        agrupar_por_valor()
    elif informacao == 'ano':
        agrupar_por_ano()
    elif informacao == 'data':
        agrupar_por_data()
    else:
        print("Opção inválida. Tente novamente.")

In [ ]:
def agrupar_por_tipo():
    agrupamento = {}

    for registro in registros:
        chave = registro['tipo']

        if chave not in agrupamento:
            agrupamento[chave] = 0

        agrupamento[chave] += registro['valor']

    print("\nAgrupado por Tipo:")
    for chave, total in agrupamento.items():
        print(f"{chave.capitalize()}: R$ {total:.2f}")
    print()

In [ ]:
def agrupar_por_data():
    agrupamento = {}

    for registro in registros:
        chave = registro['data']

        if chave not in agrupamento:
            agrupamento[chave] = 0

        agrupamento[chave] += registro['valor']


    print("\nAgrupado por Data:")
    for chave, total in agrupamento.items():
        print(f"{chave}: R$ {total:.2f}")
    print()

In [ ]:
def agrupar_por_valor():
    agrupamento = {}

    for registro in registros:
        chave = registro['valor']

        if chave not in agrupamento:
            agrupamento[chave] = 0

        agrupamento[chave] += registro['valor']


    print("\nAgrupado por Valor:")
    for chave, total in agrupamento.items():
        print(f"R$ {chave:.2f}: R$ {total:.2f}")
    print()


In [ ]:
def agrupar_por_mes():
    mes_desejado = int(input("Digite o mês desejado (1-12): "))

    agrupamento = {}

    for registro in registros:
        if registro['mes'] == mes_desejado:
            chave = registro['tipo']

            if chave not in agrupamento:
                agrupamento[chave] = 0

            agrupamento[chave] += registro['valor']


    print(f"\nAgrupado por Tipo no Mês {mes_desejado}:")
    for chave, total in agrupamento.items():
        print(f"{chave.capitalize()}: R$ {total:.2f}")
    print()


In [ ]:
def agrupar_por_ano():
    agrupamento = {}

    for registro in registros:
        chave = registro['ano']

        if chave not in agrupamento:
            agrupamento[chave] = 0

        agrupamento[chave] += registro['valor']


    print("\nAgrupado por Ano:")
    for chave, total in agrupamento.items():
        print(f"{chave}: R$ {total:.2f}")


In [ ]:
def menu():
    apresentacao_1 = apresentacao(data_hora()['hora'])
    print(f"{apresentacao_1} Bem Vindo ao sistema Ada")
    print('Escolha a operação que deseja realizar !')

    while True:
        print("Menu:")
        print("1. Criar Registro")
        print("2. Ler Registros")
        print("3. Atualizar Registro")
        print("4. Deletar Registro")
        print("5. Exportar Relatório")
        print("6. Atualizar Rendimento")
        print("7. Agrupar por")
        print("8. Sair")

        opcao = input("Escolha uma opção (1-8): ")

        if opcao == '1':
            tipo = input("Tipo (Receita/Despesa/Investimento): ").lower()
            valor = float(input("Valor: "))
            criar_registro(tipo, valor)

        elif opcao == '2':
            ler_registros()

        elif opcao == '3':
            registro_id = int(input("ID do Registro a ser Atualizado: "))
            novo_valor = float(input("Novo Valor: "))
            novo_tipo = input("Novo Tipo: ")
            nova_data = input("Nova Data (DD/MM/AAAA HH:MM): ")
            atualizar_registro(registro_id, novo_valor, novo_tipo, nova_data)

        elif opcao == '4':
            registro_id = int(input("ID do Registro a ser Deletado: "))
            deletar_registro(registro_id)

        elif opcao == '5':
            formato = input("Formato de Exportação (csv/json): ").lower()
            exportar_relatorio(formato)

        elif opcao == '6':
            atualiza_rendimento()
            print("Rendimento atualizado com sucesso.")

        elif opcao == '7':
            informacao = input("Agrupar por (mes/tipo/valor/data): ").lower()
            if informacao in ['mes', 'tipo', 'valor','data']:
                agrupar_por(informacao)
            else:
                print("Opção inválida. Tente novamente.")

        elif opcao == '8':
            print("Saindo do Sistema.")
            break

        else:
            print("Opção inválida. Tente novamente.")

In [ ]:
registro_ficticio = gerar_registro_ficticio()
menu()

Boa Noite! Bem Vindo ao sistema Ada
Escolha a operação que deseja realizar !
Menu:
1. Criar Registro
2. Ler Registros
3. Atualizar Registro
4. Deletar Registro
5. Exportar Relatório
6. Atualizar Rendimento
7. Agrupar por
8. Sair
Escolha uma opção (1-8): 8
Saindo do Sistema.
